# BASE to NER

by Andreas Sünder & Benjamin Kissinger

Das Ziel dieses Notebooks ist ein normales Dataset zu tokenizen.

Als erstes importieren wir alle notwendigen Libraries

In [1]:
import json

import nltk
from datasets import load_dataset
from nltk.tokenize import word_tokenize

nltk.download("punkt", quiet=True) # Spezieller Tokenizer

/home/ben10/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

Anschließend speichern wir das Dataset in ein Python Objekt. In diesem Fall importieren wir es von HuggingFace, jedoch kann man es auch lokal laden. Zusätzlich definieren wir gleich, welchen NER-Tag wir welcher Zahl zuweisen wollen. In diesem Fall wollen wir Autoren und Daten erkennen. Daher definieren wir: O==>0; Author==>1; Date==>2

In [2]:
dataset = load_dataset("textminr/ner_extended")
ner_tag_to_id = {"O": 0, "AUTHOR": 1, "DATE": 2}

Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 713.68it/s]
Generating train split: 6361 examples [00:00, 202228.21 examples/s]
Generating validation split: 1591 examples [00:00, 195366.62 examples/s]


Die anschließende Funktion erledigt prinzipell die ganze Arbeit. Im Anschluss wird diese genauer erklärt:

In [3]:
def convert_row(row):
    prompt = row["prompt"]
    response = json.loads(row["response"])
    author = word_tokenize(response["author"])
    date = response["date"]

    tokens = word_tokenize(prompt)
    ner_ids = []
    ner_tags = []

    counter_author = 0
    counter_date = 0

    for token in tokens:
        if token in author and counter_author < len(author):
            ner_ids.append(ner_tag_to_id["AUTHOR"])
            ner_tags.append("AUTHOR")
            counter_author += 1
        elif token in date and counter_date < 1:
            ner_ids.append(ner_tag_to_id["DATE"])
            ner_tags.append("DATE")
            counter_date += 1
        else:
            ner_ids.append(ner_tag_to_id["O"])
            ner_tags.append("O")

    row["tokens"] = tokens
    row["ner_ids"] = ner_ids
    row["ner_tags"] = ner_tags

    return row

dataset_tokenized = dataset.map(convert_row).remove_columns(["prompt", "response"])

Map: 100%|██████████| 1591/1591 [00:01<00:00, 1418.01 examples/s]


Ein Entry aus dem Ursprungsdatensatz ist wie folgt aufgebaut:

*{"prompt": "This text was written by Robert Smith on the 4th of July, 2020.", "response": "{\"author\": \"Robert Smith\", \"date\": \"2020\"}"}*

Im Prinzip müssen wir jetzt 2 Listen erstellen, welche jeweils entweder die NER_IDS oder die NER_TAGS speichern.

Wir speichern nun alle Elemente dieses Datasets in einer Variable und zusätzlich tokenizen wir den Autor und den Prompt. Das Datum müssen wir nicht tokenizen, da dieses immer nur aus einer Jahreszahl besteht und ergo ein Element ist. 

Nun gehen wir die ganzen Tokens des Prompts durch und setzen jeden Token auf 1, wenn dieser im Autor vorkommt, jeden Token auf 2, wenn dieser im Date vorkommt und alle anderen werden auf 0 gesetzt, da diese für dieses Modell irrelevant sind (weil wir eben nur Autoren und Daten klassifizieren wollen). Die erstellten Liste fügen wir jetzt als neue Spalten pro Zeile ein. 

Den Rest erledigt die **map**-Methode für uns. Diese wendet die mitgegebene Methode für alle Reihen eines datasets an. Also wird der oben beschriebene Prozess an allen Zeilen angewandt. Zusätzlich werden die Spalten "prompt" und "response" gelöscht, da wir diese nicht mehr für den NER-Classifier benötigen. 

Anschließend pushen wir den neuen Datensatz noch auf HuggingFace (optional, nicht vergessen den Token zu setzen).